In [ ]:
!pip install git+https://github.com/huggingface/nlp.git
!pip install pyarrow==0.16.0
!pip install transformers
!git clone https://github.com/chrisliu298/tapt.git

In [1]:
import os
os.chdir("/content/tapt/src")

In [2]:
import pandas as pd
from pprint import pprint
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

from utils.data_pipeline import prepare_data
from utils.data_pipeline import prepare_custom_data
from utils.metrics import compute_metrics


def tokenize(batch):
    """Tokenize a batch of data (with padding and truncation).

    Arg:
        batch: A batch of training data.
    """
    return tokenizer(
        batch["text"], padding="max_length", truncation=True, max_length=512
    )

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
# Load model
# model = AutoModelForSequenceClassification.from_pretrained(
#     "distilroberta-base"
# )

# Load dataset
train_dataset, val_dataset, test_dataset = prepare_data(
    tokenize_func=tokenize,
    dataset_name="yelp_polarity",
    train_count=10000,
    train_size=8000,
    val_size=2000,
    test_count=10000,
    test_size=5000,
    non_test_size=5000,
    use_all_test=False,
    seed=42,
)
# Load custom data
augmented = prepare_custom_data(
    tokenize_func=tokenize, dataset_name="/content/nlp_yelp_train.tsv"
)

Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/4ec39419abf6679dd9d28ce58339ab3247c834b234c215acfa8dcc9dd771aab4. Subsequent calls will reuse this data.


In [3]:
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))
print(len(augmented))

8000
2000
5000
25000


In [13]:
pprint(train_dataset.features)

{'attention_mask': [Value(dtype='int64', id=None)],
 'input_ids': [Value(dtype='int64', id=None)],
 'label': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None)}


In [14]:
pprint(train_dataset.num_columns)

4


In [15]:
pprint(train_dataset.num_rows)

8000


In [8]:
train_dataset["text"][:5]

["I honestly wouldn't give this place any star's if that was possible. It's been awhile since I've had absolutely horrible service and tasteless food all in one visit. But this location seemed to hit both.\\n\\nThe service was HORRIBLE. My friend and I decided to sit at the bar, thinking that getting drinks would be easier but the girl was for lack of better words just out of it and not paying attention and we had to wave her down every time we wanted to get something to drink.\\n\\nAs for the food. THE WORST sushi I've ever had. The rolls were seriously overpriced, and disgusting.Aside from the fact she just completely forgot two of our rolls. Our meals came with soup that our waitress/bartender forgot about and she but after we reminded her she brought them out and it was basically just broth no tofu. Totally cheap'd out on the Miso Soup. \\n\\nI left that place soo upset and just soo disappointed. I would STRONGLY suggest you not waste your time nor your money on making a visit here

In [9]:
train_dataset["label"][:5]

array([0, 0, 1, 1, 1])